In [ ]:
!pip install contractions
!pip install emoji
!pip install ekphrasis
!pip install -U -q PyDrive
!pip install yfinance --upgrade --no-cache-dir

import contractions
import datetime
from datetime import datetime , timedelta, date
import emoji
import itertools
import json
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer, word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import numpy as np
import os
import pandas as pd
import pprint
from pprint import pprint
import random
import re
import requests 
import seaborn as sns
import string
import sys
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import torch
import yfinance as yf
from ekphrasis.classes.segmenter import Segmenter
from pydrive.auth  import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth,  drive
from oauth2client.client import GoogleCredentials
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

drive.mount('/content/drive')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
pb = '/content/drive/My Drive/StockTwits Data Binary/'
pp = '/content/drive/My Drive/StockTwits Data Percentage/'
l1 = 'label2.1.csv'
l2 = 'label2.2.csv'
binary_same = [pb+'HD_binary_'+l2,pb+'T_binary_'+l2,pb+'GOOG_binary_'+l2,pb+'BRK.A_binary_'+l2]
binary_prev = [pb+'HD_binary_'+l1,pb+'T_binary_'+l1,pb+'GOOG_binary_'+l1,pb+'BRK.A_binary_'+l1]
percentage_same = [pp+'HD_'+l2,pp+'T_'+l2,pp+'PG_'+l2,pp+'GOOG_'+l2,pp+'BRK.A_'+l2]
percentage_prev = [pp+'HD_'+l1,pp+'T_'+l1,pp+'PG_'+l1,pp+'GOOG_'+l1,pp+'BRK.A_'+l1]
method = [binary_same,binary_prev,percentage_same,percentage_prev]
y = 0
for x in method:
  li = []
  for filename in method[y]:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
  df = pd.concat(li, axis=0, ignore_index=True)
  df.to_csv('combined_'+str(y)+'.csv', index=False)
  y = y+1
binary_same = pd.read_csv('/content/combined_0.csv')
binary_prev = pd.read_csv('/content/combined_1.csv')
percentage_same = pd.read_csv('/content/combined_2.csv')
percentage_prev = pd.read_csv('/content/combined_3.csv')
binary_same.to_csv('/content/drive/MyDrive/HD_T_GOOG_BRK-A_TSLA_Binary_Sameday.csv', header=True, index=False, encoding='utf_8')
binary_prev.to_csv('/content/drive/MyDrive/HD_T_GOOG_BRK-A_TSLA_Binary_Prevday.csv', header=True, index=False, encoding='utf_8')
percentage_same.to_csv('/content/drive/MyDrive/HD_T_GOOG_BRK-A_TSLA_Percentage_Sameday.csv', header=True, index=False, encoding='utf_8')
percentage_prev.to_csv('/content/drive/MyDrive/HD_T_GOOG_BRK-A_TSLA_Percentage_Prevday.csv', header=True, index=False, encoding='utf_8')
binary_same.drop(binary_same[binary_same['Date'] <= '2019-07-20'].index, inplace = True)
binary_same.reset_index(drop=True, inplace=True)
binary_prev.drop(binary_prev[binary_prev['Date'] <= '2019-07-20'].index, inplace = True)
binary_prev.reset_index(drop=True, inplace=True)
percentage_same.drop(percentage_same[percentage_same['Date'] <= '2019-07-20'].index, inplace = True)
percentage_same.reset_index(drop=True, inplace=True)
percentage_prev.drop(percentage_prev[percentage_prev['Date'] <= '2019-07-20'].index, inplace = True)
percentage_prev.reset_index(drop=True, inplace=True)
print('Binary same day value counts:')
print(binary_same['label'].value_counts())
print()
print('Binary prev day value counts:')
print(binary_prev['label'].value_counts())
print()
print('Percentage same day value counts:')
print(percentage_same['label'].value_counts())
print()
print('Percentage prev day value counts:')
print(percentage_prev['label'].value_counts())

In [ ]:
print('binary_same')
plt.figure(figsize = (9,3))
ax = plt.pie(binary_same['label'].value_counts(), colors = colors, autopct="%1.1f%%", labels=["Positive","Negative"])

In [ ]:
print('binary_prev')
plt.figure(figsize = (9,3))
ax = plt.pie(binary_prev['label'].value_counts(), colors = colors, autopct="%1.1f%%", labels=["Positive","Negative"])

In [ ]:
print('percentage_same')
colors = ['cornflowerblue', 'antiquewhite', 'lightcoral']
plt.figure(figsize = (9,3))
ax = plt.pie(percentage_same['label'].value_counts(), colors = colors, autopct="%1.1f%%", labels=["Positive","Netrual","Negative"])

In [ ]:
print('percentage_prev')
colors = ['antiquewhite', 'cornflowerblue', 'lightcoral']
plt.figure(figsize = (9,3))
ax = plt.pie(percentage_prev['label'].value_counts(), colors = colors, autopct="%1.1f%%", labels=["Netrual","Positive","Negative"])

In [ ]:
df['symbol'].value_counts()

In [ ]:
plt.figure(figsize = (9,3))
plt.ticklabel_format(style='plain', axis='x')
ax = sns.countplot(y="symbol", data=df, palette=("coolwarm"), order = df['symbol'].value_counts().index)
ax.set(xlabel='Number of StockTwits messages', ylabel='')

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd
comment_words = ''
set1 = set(STOPWORDS)
set2 =  {"aapl","adbe","amzn","bac","brk.a","brk.b","brk","dia","dis","fb","goog","googl","hd","intc","jnj","nflx","pg","qqq","spy","t","tsla","unh","v","vix","vz","wmt","quot","home","depot","google","amp"}
stopwords = set.union(set1, set2)
for val in df.message:
    val = str(val)
    tokens = val.split()     
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()  
    comment_words += " ".join(tokens)+" "
wordcloud = WordCloud(width = 2400, height = 1600, background_color ='white',stopwords = stopwords, min_font_size = 5).generate(comment_words)
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
result = {"Model":['Logistic Reggresion','Navis Bayes','Random Forest','BERT'],"Time":[33,65,100,185]}
df_results = pd.DataFrame(result) 
plt.figure(figsize = (9,3))
plt.ticklabel_format(style='plain', axis='x')
sns.barplot(x=df_results['Model'], y=df_results['Time'], palette=("coolwarm"))
plt.title("Model running time in minuttes")